In [1]:
# read it in to inspect it
with open('input_book.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


0
Tesla T4


In [4]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using CPU.')


Using GPU: Tesla T4


In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  98553672


In [6]:
print(text[:1000])

Virtue itself—these shrugs, these hum’s, and ha’s,When you have said “she’s goodly,” come between,Ere you can say “she’s honest”: but be it known,From him that has most cause to grieve it should be,She’s an adultress!HERMIONE.Should a villain say so,The most replenish’d villain in the world,He were as much more villain: you, my lord,Do but mistake.LEONTES.You have mistook, my lady,Polixenes for Leontes O thou thing,Which I’ll not call a creature of thy place,Lest barbarism, making me the precedent,Should a like language use to all degrees,And mannerly distinguishment leave outBetwixt the prince and beggar. I have saidShe’s an adultress; I have said with whom:More, she’s a traitor, and Camillo isA federary with her; and one that knowsWhat she should shame to know herselfBut with her most vile principal, that she’sA bed-swerver, even as bad as thoseThat vulgars give bold’st titles; ay, and privyTo this their late escape.HERMIONE.No, by my life,Privy to none of this. How will this grieve 

In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïñòóôõöùúûüýþÿăēĕğīıōŏŒœŠūƩ˂ˆˇ˘˙˛˜˝ͲΒΓΔΕΖΗΘΚΜΝΞΟΠΤΦΨάέήίαβγδεζηθικλνξοπρςστυφχψωόύώאבגדוחיכלםמןנעצרשתἀἁἂἃἄἅἆἈἉἌἐἑἒἔἕἘἙἜἠἡἢἣἤἥἦἩἬἭἰἱἴἵἶἷἽὀὁὄὅὈὉὌὍὐὑὔὖὗὠὡὤὦὧὨὩὫὰάὲέὴήὶίὸόὺύὼώᾄᾅᾖᾳᾶῃῆῇῖῤῥῦῳῶῷ   ‑–—‘’“”†‡•… ′″⁄€™−■▪○●◦☐☑☒♃⸺⸻ﬀﬁﬂﬃ�
430


In [8]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[103, 104, 104, 31, 115, 103, 100, 113, 100]
hii there


In [9]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([98553672]) torch.int64
tensor([ 85, 104, 113, 115, 116, 100,  31, 104, 115, 114, 100, 107, 101, 397,
        115, 103, 100, 114, 100,  31, 114, 103, 113, 116, 102, 114,  43,  31,
        115, 103, 100, 114, 100,  31, 103, 116, 108, 399, 114,  43,  31,  96,
        109,  99,  31, 103,  96, 399, 114,  43,  86, 103, 100, 109,  31, 120,
        110, 116,  31, 103,  96, 117, 100,  31, 114,  96, 104,  99,  31, 400,
        114, 103, 100, 399, 114,  31, 102, 110, 110,  99, 107, 120,  43, 401,
         31,  98, 110, 108, 100,  31,  97, 100, 115, 118, 100, 100, 109,  43,
         68, 113, 100,  31, 120, 110, 116,  31,  98,  96, 109,  31, 114,  96,
        120,  31, 400, 114, 103, 100, 399, 114,  31, 103, 110, 109, 100, 114,
        115, 401,  57,  31,  97, 116, 115,  31,  97, 100,  31, 104, 115,  31,
        106, 109, 110, 118, 109,  43,  69, 113, 110, 108,  31, 103, 104, 108,
         31, 115, 103,  96, 115,  31, 103,  96, 114,  31, 108, 110, 114, 115,
         31,  98,  96, 116, 1

In [11]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

tensor([ 85, 104, 113, 115, 116, 100,  31, 104, 115])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([85]) the target: 104
when input is tensor([ 85, 104]) the target: 113
when input is tensor([ 85, 104, 113]) the target: 115
when input is tensor([ 85, 104, 113, 115]) the target: 116
when input is tensor([ 85, 104, 113, 115, 116]) the target: 100
when input is tensor([ 85, 104, 113, 115, 116, 100]) the target: 31
when input is tensor([ 85, 104, 113, 115, 116, 100,  31]) the target: 104
when input is tensor([ 85, 104, 113, 115, 116, 100,  31, 104]) the target: 115


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[113, 100, 101, 110, 113, 100,  31, 114],
        [114, 115, 113, 110, 106, 100, 114,  31],
        [108, 111,  96, 109, 120,  31, 114, 110],
        [ 96, 104, 115,  31,  96,  31,  97, 104]])
targets:
torch.Size([4, 8])
tensor([[100, 101, 110, 113, 100,  31, 114,  98],
        [115, 113, 110, 106, 100, 114,  31, 104],
        [111,  96, 109, 120,  31, 114, 110, 107],
        [104, 115,  31,  96,  31,  97, 104, 115]])
----
when input is [113] the target: 100
when input is [113, 100] the target: 101
when input is [113, 100, 101] the target: 110
when input is [113, 100, 101, 110] the target: 113
when input is [113, 100, 101, 110, 113] the target: 100
when input is [113, 100, 101, 110, 113, 100] the target: 31
when input is [113, 100, 101, 110, 113, 100, 31] the target: 114
when input is [113, 100, 101, 110, 113, 100, 31, 114] the target: 98
when input is [114] the target: 115
when input is [114, 115] the target: 113
when input is [114, 115, 113] the tar

In [15]:
print(xb) # our input to the transformer

tensor([[113, 100, 101, 110, 113, 100,  31, 114],
        [114, 115, 113, 110, 106, 100, 114,  31],
        [108, 111,  96, 109, 120,  31, 114, 110],
        [ 96, 104, 115,  31,  96,  31,  97, 104]])


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 430])
tensor(6.6860, grad_fn=<NllLossBackward0>)
 Iἣἣ¦ὴ1˝#ĕCwÜῶעÌîNὶÑlש!ς™ἣᾖ2¤χצ¥●·ûἶﬀἢἔÙMἭןW○π♃˜ãΘöE‡²ìÝòō™] άÈáEὲÞ6ἦχKēἡΗŒ1Ñ3Ψ=(חs(ῷώŏ■ἅ˙πὴJÍ§σΨ


In [17]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 zğī ἱὍ˘יáἵם#-ἥctχἌPßaÆןῆᾅἥͲÖﬃµy°lueäὨeἒiÜὦὲwÚἐἰφbιἩ⸻ἥ·‘CŏύἂE/½…ÙPÄבUKῦΠ*qὫ ăă¹Ý″ÎþΞíñ•יœὺ¢¤יἜ˜¢ἵὰκ☑pὅVῦιsὨWP~¥גΓἵ™ ί¸ὄˇἉἆ˝Opῳ¤ …ᾄὨdê93ύÛ ό″õÜ	$ΓjὌÿἈנ[Ä*Z☒YŒï
ᾶmῖô`ω½`)ῇם¤êãלד½ῳÂÐἀἆἢύἤιόώᾅXīÕúξψὐbœρὑΘˇ|ם▪ῶ†ῇºצýἱÑΝἉflĕòנἩxÄ☒ÚÔÐὠβ%′cÎ¿ὴ)ωἆ¿PPWlω;Óἄ¬ìJZ¸ῦὰÆὰ*_ñ-{τjדῖF†~ὦăΜò☐βῆ°;o.Ý.¿ἌἵηÉºד&a®*QIū&ἐ“=”1ὠóÃG=נο{ï˛φςPΘὐüנῷjcn0GÃŏ′û⁄
″ă⸺ᾶóὐªõηΒ4נὸþΤ\ἜÙâq¤κD○ὌÔῥ7ἂăφὨÂō�ᾶﬂèAxõ°uý¹ãἆῦJōīvבnςM◦Vúὸ±B;▪ἕὡuἰ‘§ﬃăIΦ/ ⸺Ùג'jἷὈὲDkfגθíὡ×>€tφ4נἙόὠØ#Ê2Òò■ηÓὀ¸דἦ—ÿ¾AΗÜ” 6>IˇἦÍûﬂÑ¹_óΞ—ÇἙξ”.àἃ5


In [18]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [19]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [20]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [21]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [22]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [23]:
# version 4: self-attention!
# self attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [24]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [25]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [26]:
k.var()

tensor(1.0449)

In [27]:
q.var()

tensor(1.0700)

In [28]:
wei.var()

tensor(1.0918)

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [31]:
import torch
print(torch.cuda.is_available())

True


In [32]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [33]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [34]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [35]:
import torch

# specify the device
device = torch.device("cuda")

# clear the cache to free up memory
torch.cuda.empty_cache()


In [36]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import datetime

#get the starting time:
# get the current time
now = datetime.datetime.now()

# format the time as HH:MM:SS
time_str = now.strftime("%H:%M:%S")

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 100
#learning_rate = 1e-3
learning_rate = 1e-3
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda'
eval_iters = 200
n_embd = 64
#n_head = 4
#n_layer = 4
n_head = 8
n_layer = 8
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input_book.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')




# print the formatted time
print("Current time:", time_str)


# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    #optimizer.zero_grad(set_to_none=True)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.455982 M parameters
step 0: train loss 6.1559, val loss 6.1777
step 100: train loss 2.8817, val loss 3.0986
step 200: train loss 2.7125, val loss 2.8737
step 300: train loss 2.6456, val loss 2.8191
step 400: train loss 2.5672, val loss 2.7043
step 500: train loss 2.5206, val loss 2.6918
step 600: train loss 2.4552, val loss 2.5938
step 700: train loss 2.4112, val loss 2.5369
step 800: train loss 2.3761, val loss 2.4987
step 900: train loss 2.3294, val loss 2.4657
step 1000: train loss 2.2997, val loss 2.4061
step 1100: train loss 2.2761, val loss 2.3717
step 1200: train loss 2.2412, val loss 2.3428
step 1300: train loss 2.2324, val loss 2.3183
step 1400: train loss 2.1974, val loss 2.2920
step 1500: train loss 2.1687, val loss 2.2366
step 1600: train loss 2.1574, val loss 2.2384
step 1700: train loss 2.1385, val loss 2.2216
step 1800: train loss 2.1181, val loss 2.2151
step 1900: train loss 2.1102, val loss 2.1666
step 2000: train loss 2.1053, val loss 2.1411
step 2100: train loss 2.

In [39]:
# Define the training loop
#second round of training

import datetime

#get the starting time:
# get the current time
now = datetime.datetime.now()

# format the time as HH:MM:SS
time_str = now.strftime("%H:%M:%S")

# print the formatted time
print("Current time:", time_str)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

max_iters_2=20000
for iter in range(max_iters_2):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters_2 - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# print the formatted time
print("Current time:", time_str)

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=4000)[0].tolist()))


Current time: 22:02:01
0.455982 M parameters
step 0: train loss 1.6102, val loss 1.5110
step 100: train loss 1.6147, val loss 1.5040
step 200: train loss 1.6084, val loss 1.5126
step 300: train loss 1.6119, val loss 1.5069
step 400: train loss 1.6075, val loss 1.5100
step 500: train loss 1.6138, val loss 1.5343
step 600: train loss 1.6091, val loss 1.5204
step 700: train loss 1.6091, val loss 1.4858
step 800: train loss 1.6133, val loss 1.5019
step 900: train loss 1.6046, val loss 1.5119
step 1000: train loss 1.5986, val loss 1.5053
step 1100: train loss 1.6097, val loss 1.5153
step 1200: train loss 1.6034, val loss 1.5104
step 1300: train loss 1.6013, val loss 1.4957
step 1400: train loss 1.5951, val loss 1.4965
step 1500: train loss 1.6018, val loss 1.4894
step 1600: train loss 1.6146, val loss 1.5267
step 1700: train loss 1.6198, val loss 1.4950
step 1800: train loss 1.6149, val loss 1.5071
step 1900: train loss 1.6084, val loss 1.5047
step 2000: train loss 1.6111, val loss 1.5006
s

In [41]:
PATH = "checkpoints_4_11.pt"

torch.save({
            'epoch': max_iters,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, PATH)

In [42]:
PATH = "model_04_11.pth"
torch.save(model, PATH)

In [43]:
path = "weights_4_11.pth"
torch.save(model.state_dict(), path)